In [ ]:
import os
import pandas as pd

##initialize first dataframe
df = pd.read_excel('combinepdf_clean.xlsx')

##name1 = df.columns[5]
##name2 = df.columns[6]
##print(name1)
##new_names = {df.columns[5]: 'Person1', df.columns[6]: 'Person2'}
##df = df.rename(columns=new_names)
##team = name1 + ", " + name2
# insert a new column at the beginning of the DataFrame
##new_column = pd.Series([name1 for i in range(5)], name='Name1')
##df.insert(0, 'Name1', new_column)
##new_column = pd.Series([name2 for i in range(5)], name='Name2')
##df.insert(1, 'Name2', new_column)
##print(df)

# Remove unused metric of tournament totals
mask = ~df['R'].astype(str).str.startswith('Tournament Totals:')
df = df.loc[mask]
# Remove Team names as not needed
mask = ~df['R'].astype(str).str.startswith('Team:')
df = df.loc[mask]

##Drop random column
df = df.drop(columns =['Unnamed: 8'])



#Remove repeat columns from aggregation
# mask = ~df['R'].astype(str).str.startswith('R')
# df = df.loc[mask]
## drop rows with null values
df = df.dropna(how='any')
# df = df.iloc[1:]

    
output_path = "./file.csv"
df.to_csv(output_path, index=False)
##IMPORTANT LOOK AT THESE MANUAL INSTRUCTIONS BELOW
##MANUALLY REMOVE HEADER ROW HERE BEFORE PRE-Proce
##Delete 5474-5564 in the resulting file, weird data that is inconsistent with everything else
##could include this code below


In [34]:
import csv

with open('file.csv', 'r', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    
    ##Dictionary defines names as keys and a value of a 'list' with specified measurements
    ##[total rounds, total speaks, total ranks, average speaks, average ranks, overall win/loss ratio, gov win/loss ratio, opp. win/loss ratio]
    ##during parsing, overall win/loss and the indices onward will hold its own list to hold the needed intermediate values for final calc. at end
    db = dict()
    row = next(csvreader)
    p = 1
    while True:
        try:
            if row[0] == 'R':
                name1 = row[5]
                if db.get(name1) == None:
                    db.update({name1: [0, 0, 0, 0, 0, [0 , 0], [0, 0], [0, 0]]})
                    db[name1][0] = 0
                    # ##update ratio indicies with temporary list for final calc.
                    # db[name1][5] = [0, 0]
                    # db[name1][6] = [0, 0]
                    # db[name1][7] = [0, 0]
                
                name2 = row[6]
                if db.get(name2) == None:
                    db.update({name2: [0, 0, 0, 0, 0, [0 , 0], [0, 0], [0, 0]]})
                    db[name2][0] = 0
                    # ##update ratio indicies with temporary list for final calc.
                    # db[name2][5] = [0, 0]
                    # db[name2][6] = [0, 0]
                    # db[name2][7] = [0, 0]
                    
                
                ##iterate actual table information
                row = next(csvreader)
                p+=1
                while row[0] != 'R':
                    ##update total rounds
                    db[name1][0] +=1
                    db[name2][0] +=1

                    ##update total speaks and total ranks
                    speak_rank1 = row[5][1:-1]
                    speak_rank2 = row[6][1:-1]
                    
                    ##parse the strings further,
                    sprlist1 = speak_rank1.split(",")
                    sprlist2 = speak_rank2.split(",")
                    ##convert them to integers
                    sprlist1 = [float(x) for x in sprlist1]
                    sprlist2 = [float(x) for x in sprlist2]


                    ##update total speaks and total ranks with the extracted data
                    ##speaks
                    db[name1][1]+=sprlist1[0]
                    db[name2][1]+=sprlist2[0]

                    ##ranks
                    db[name1][2]+=sprlist1[1]
                    db[name2][2]+=sprlist2[1]

                    ##update intermediate win and loss
                    if row[2] == 'W':
                        db[name1][5][0]+=1
                        db[name2][5][0]+=1
                        if row[1] == 'G':
                            ##update gov win
                            db[name1][6][0]+=1
                            db[name2][6][0]+=1
                        else:
                            ##update opp win
                            db[name1][7][0]+=1
                            db[name2][7][0]+=1
                    else:
                        db[name1][5][1]+=1
                        db[name2][5][1]+=1
                        if row[1] == 'G':
                            ##update gov loss
                            db[name1][6][1]+=1
                            db[name2][6][1]+=1
                        else:
                            ##update opp loss
                            db[name1][7][1]+=1
                            db[name2][7][1]+=1
                    ##next row
                    row = next(csvreader)
                    p+=1
                
        except StopIteration:
            # Reached the end of the file
            break
    
    ##print(db)
        
        
    ##update the average metrics with total rounds and other fields, and win/loss ratios as well
    for entry in db:
        ##average speaks
        try:
            db[entry][3] = round(db[entry][1]/db[entry][0], 2)
        except ZeroDivisionError:
            db[entry][3] = round(db[entry][1], 2)
        
        ##average ranks
        try:
            db[entry][4] = round(db[entry][2]/db[entry][0], 2)
        except:
            db[entry][4] = round(db[entry][2], 2)
        
        ##overall win/loss ratio
        try:
            db[entry][5] = round(db[entry][5][0]/db[entry][5][1], 2)
        except:
            db[entry][5] = round(db[entry][5][0], 2)
        ##gov win/loss ratio
        try:
            db[entry][6] = round(db[entry][6][0]/db[entry][6][1], 2)
        except ZeroDivisionError:
            db[entry][6] = round(db[entry][6][0], 2)
        
        ##opp win/loss ratio
        try: 
            db[entry][7] = round(db[entry][7][0]/db[entry][7][1], 2)
        except ZeroDivisionError:
            db[entry][7] = round(db[entry][7][0], 2)
      
                                
    print(db)

{'Bart Kaminski (V)': [15, 455.0, 29.0, 30.33, 1.93, 1.5, 1.67, 1.33], 'Eugenie Park (V)': [54, 1619.0, 105.0, 29.98, 1.94, 1.7, 2.38, 1.25], 'Andrew Monteith (V)': [34, 996.0, 68.0, 29.29, 2.0, 1.12, 1.38, 0.88], 'Peregrine Beckett (V)': [5, 141.0, 13.0, 28.2, 2.6, 1.5, 0.5, 2], 'Irina Znamirowski (V)': [30, 872.0, 71.0, 29.07, 2.37, 1.14, 0.88, 1.5], 'John Lowe (V)': [5, 143.0, 14.0, 28.6, 2.8, 0.67, 0.0, 2], 'Ahmad Howard (V)': [49, 1491.0, 83.0, 30.43, 1.69, 2.77, 2.86, 2.67], 'Helen W (V)': [5, 145.0, 12.0, 29.0, 2.4, 1.5, 1.0, 2.0], 'Aidan Shannon (V)': [40, 1133.0, 93.0, 28.32, 2.33, 0.9, 1.0, 0.83], 'Nish Sinha (V)': [30, 845.0, 84.0, 28.17, 2.8, 1.0, 0.78, 1.33], 'Katrina Hotaling (V)': [55, 1711.0, 102.0, 31.11, 1.85, 3.58, 2.88, 5.0], 'Nick DeVito (V)': [38, 1202.0, 60.0, 31.63, 1.58, 3.75, 2.67, 7.0], 'Gabe Ritter (V)': [15, 461.0, 31.0, 30.73, 2.07, 4.0, 3.5, 5.0], 'Prottoya Chowdhury (V)': [10, 276.0, 32.0, 27.6, 3.2, 1.5, 4.0, 0.67], 'Leo Fleissig (V)': [44, 1261.0, 103.

In [2]:
##Algorithm Decision Tree implementation
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

##NOT UPDATED

##target variable
var = df['W/L']

names = ['R', 'G/O', 'Opponent', 'Judge', 'Person1', 'Person2', 'Total']

print(df[names])

##properly create dataframe, transformming
X = pd.get_dummies(df[names])


# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(
            X, var, test_size = 0.25, random_state=42)

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)

##Predict and evaluate
y_pred = tree.predict(X_test)


print("Accuracy of Tree:", metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))
print("")


tree_rules = export_text(tree, feature_names=list(X_train))
print("")
print(tree_rules)




KeyError: 'W/L'